<a href="https://colab.research.google.com/github/coltongerth/RPMS-Cutter/blob/main/Copy_of_rpms_cutout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

These are some libraries that will need to be downloaded if the next cell throws some ModuleNotFound errors. Delete the # of the line who is missing and run the cell.

In [ ]:
%%capture
!pip install rasterio
!pip install rioxarray
!pip install ipyleaflet
!pip install dask
!pip install dask[distributed]
!pip install xarray

# **Library Imports and Back End Setup**
Nothing needs to be changed here, this cell just imports all the libraries our code needs to run, and gets Google Drive mounted to this notebook.

You **will** need to approve this notebook to having access to your local Google Drive, it will be a little pop up when this cell gets run!

In [ ]:
import os
import datetime
import numpy as np
import xarray as xr
import pandas as pd
import rasterio
import rioxarray
import dask
import tempfile
from rasterio.coords import BoundingBox
from ipyleaflet import *
from google.cloud import storage
from dask.distributed import Client, LocalCluster, Lock, as_completed, fire_and_forget
from google.colab import drive

drive.mount('/content/gdrive')
# cluster = LocalCluster(n_workers=8, processes=True)
# client = Client(cluster)
path_template = "https://storage.googleapis.com/fuelcast-public/rpms/"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# **Inputs**

start_year: The beginning year of the range.

end_year: The ending year of the range.

raster_file: The name of the file you dropped into the file space to the left.

**(Optional)** poly_coords: A list of coordinates. This can be typed in manually, but it is highly recommended that you use the drawing tool below.

In [ ]:
start_year = 2022
end_year = 2022
# Can look like: "degradation_bpslut4_bpslut4_wgs84.tif" if dragged and dropped on the left. OR if in gdrive already: "gdrive/MyDrive/degradation_bpslut4_bpslut4_wgs84.tif"
file_name = "NNFGbuff.tif"
prefix = "julia-test-nnfg1"
poly_coords = {"coords":[]}
nodata = -32768

# **Optional geometry drawing tool**
Use this tool if you need a slice of the incoming raster instead of the entire one. If you dont need it, you dont need to run this cell.

In [ ]:
zone_map = Map(center=(38, -97),
                zoom=5,
                basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik))

draw_control = DrawControl(
    rectangle= {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    },
    polygon={},
    polyline={},
    circlemarker={}
    )

zone_map.add_control(draw_control)

def handle_draw(self, action, geo_json):
    poly_coords["coords"] = draw_control.last_draw['geometry']['coordinates'][0]
    print(poly_coords)
    print("Done generating coordinates.")
    return poly_coords

draw_control.on_draw(handle_draw)


zone_map

Map(center=[38, -97], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

{'coords': [[-105.512695, 40.178873], [-105.512695, 46.073231], [-95.844727, 46.073231], [-95.844727, 40.178873], [-105.512695, 40.178873]]}
Done generating coordinates.


# **RPMS Extractor**
You don't need to change anything in here, just let the cell run.

In [ ]:
def pol_to_np(pol):
    """
    Receives list of coordinates: [[x1,y1],[x2,y2],...,[xN,yN]]
    """
    return np.array([list(l) for l in pol])

def pol_to_bounding_box(pol):
    """
    Receives list of coordinates: [[x1,y1],[x2,y2],...,[xN,yN]]
    """
    arr = pol_to_np(pol)
    return BoundingBox(np.min(arr[:,0]),
                       np.min(arr[:,1]),
                       np.max(arr[:,0]),
                       np.max(arr[:,1]))

with rasterio.Env():
  zone_ds = rasterio.open("/content/" + file_name)
  if poly_coords["coords"] != []:
    print("Generating boundary with drawn geometry")
    bounds = pol_to_bounding_box(poly_coords["coords"])
  else:
    print("Generating boundary with imported raster's dimensions")
    bounds = zone_ds.bounds

  profile = zone_ds.profile
  profile.update(blockxsize=1024,
                 blockysize=1024,
                 dtype = "int16",
                 nodata = nodata,
                 tiled=True)

  for y in range(start_year, end_year+1):
      dx = rasterio.open("https://storage.googleapis.com/fuelcast-public/rpms/" + str(y) + "/rpms_" + str(y) + ".tif")
      op = f"rpms_{str(y)}_mean.tif"
      with rasterio.open('/content/gdrive/MyDrive/'+ prefix + "_" + 'rpms_'+str(y)+'.tif', 'w', **profile) as dst:
        win = dx.window(bottom=bounds.bottom, right=bounds.right, top=bounds.top, left=bounds.left)
        dat = dx.read(1, window=win)
        dst.write_band(1, dat)

Generating boundary with drawn geometry
